In [1]:
import pandas as pd
import pathlib
import json

In [51]:
path_manual = "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/eval_eqs.xlsx"
path_detected = "/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/eval_eqs_pred.json"

In [52]:
df_manual = pd.read_excel(path_manual, header=None)
df_manual.columns = ["from", "to"]
df_manual

,from,to
0,alumno,alumnado
1,enfermerío,enfermería
2,desenvolvemento_rural,desarrollo_rural
3,recuperacion,recuperación
4,recubrimiento,/
...,...,...
359,canal_arlanzón,/
360,actuar,actuación
361,servicio,servicios
362,refuerzo_firme,/


In [53]:
equivalent = {}
with pathlib.Path(path_detected).open('r', encoding='utf8') as fin:
    newEq = json.load(fin)['wordlist']
newEq = [x.split(':') for x in newEq]
newEq_from = [x[0] for x in newEq if len(x) == 2]
newEq_to = [x[1] for x in newEq if len(x) == 2]
df_pred = pd.DataFrame(
    {
        "from": newEq_from,
        "to": newEq_to
    }
)
df_pred

,from,to
0,alumno,alumnado
1,enfermerío,enfermería
2,desenvolvemento_rural,desarrollo_rural
3,recuperacion,recuperación
4,recubrimiento,recuperación
...,...,...
234,pilot,piloto
235,limpiezar,limpieza
236,señalado,señalar
237,actuar,actuación


In [58]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

df_merged = df_manual.merge(df_pred, on='from', how='left', suffixes=('_manual', '_pred'))

true_mappings = []
predicted_mappings = []

for index, row in df_merged.iterrows():
    if row['to_manual'] == '/':  # Word should not be mapped
        true_mappings.append(0)  # 0 means no mapping is correct
        if pd.isna(row['to_pred']):  # If there's no prediction, it's correct
            predicted_mappings.append(0)
        else:
            predicted_mappings.append(1)  # Incorrectly predicted a mapping
    else:  # Word should be mapped
        true_mappings.append(1)  # 1 means mapping is required
        if pd.isna(row['to_pred']):  # No mapping was predicted
            predicted_mappings.append(0)
        elif row['to_manual'] == row['to_pred']:  # Correct mapping
            predicted_mappings.append(1)
        else:
            predicted_mappings.append(0)  # Incorrect mapping

precision = precision_score(true_mappings, predicted_mappings)
recall = recall_score(true_mappings, predicted_mappings)
f1 = f1_score(true_mappings, predicted_mappings)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

Precision: 0.967
Recall: 0.769
F1 Score: 0.857


In [56]:
print(f1_score(true_mappings, predicted_mappings))

0.8565488565488565


In [31]:
df_merged = df_manual.merge(df_pred, on='from', how='left', suffixes=('_manual', '_pred'))

# Filter rows where there is a valid mapping in the manual dataset (not '/')
# and either the predicted mapping is missing or incorrect
false_negatives = df_merged[
    (df_merged['to_manual'] != '/') &  # Valid mapping exists in manual annotations
    ((df_merged['to_pred'].isna()) | (df_merged['to_manual'] != df_merged['to_pred']))  # No or incorrect prediction
]

false_negatives[60:]

,from,to_manual,to_pred
353,línea_alta,línea,NaN
356,plan_recuperacion,plan_recuperación,NaN


In [32]:
# Identify cases where the "from" value in df_manual is present in the "to" column of df_pred
reverse_mappings = df_manual[df_manual['from'].isin(df_pred['to'])]
reverse_mappings

,from,to
5,deportivo,deporte
68,juegos,juego
92,provincial,provincia
93,provincial_compra,/
94,provincial_insular,/
100,recarga,/
101,dotacional,dotación
111,calefacción,/
131,plantear,/
158,agrario_tragsa,/


In [46]:
df_valid = pd.read_excel("/export/usuarios_ml4ds/lbartolome/Repos/repos_con_carlos/RAG_tool/data/cpv45_equivalences_test_annotated.xlsx")
df_valid["related_model"] = df_valid["equivalence"].apply(lambda x : 1 if x != '[]' else 0)
df_valid[df_valid["related_model"]==1]

,Unnamed: 0,origin,equivalence,related,related_model
0,0,"['alumno', 'alumnado']",[{'alumnado': ['alumno']}],1,1
1,1,"['enfermerío', 'asistente', 'enfermería']",[{'enfermería': ['enfermerío']}],1,1
2,2,"['desenvolvemento_rural', 'desarrollo_rural']",[{'desarrollo_rural': ['desenvolvemento_rural']}],1,1
3,3,"['recuperacion', 'recubrimiento', 'recuperación']","[{'recuperación': ['recuperacion', 'recubrimie...",1,1
4,4,"['deporte', 'deportivo']",[{'deportivo': ['deporte']}],1,1
...,...,...,...,...,...
240,240,"['piloto', 'pilot']",[{'piloto': ['pilot']}],1,1
241,241,"['limpieza', 'limpiezar']",[{'limpieza': ['limpiezar']}],1,1
243,243,"['señalar', 'señalado']",[{'señalar': ['señalado']}],1,1
245,245,"['actuar', 'actuacion']",[{'actuación': ['actuar']}],1,1


In [49]:
model_clusters = df_valid["related_model"].values
user_evaluation =  df_valid["related"].values

In [57]:
precision = precision_score(user_evaluation, model_clusters)
recall = recall_score(user_evaluation, model_clusters)
f1 = f1_score(user_evaluation, model_clusters)

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')

Precision: 0.968
Recall: 0.772
F1 Score: 0.859
